# Day 4 - Working With Files and Calling APIs

Welcome to Day 4 of our Python course. Today we will:

- Get comfortable with **files and directories** on your computer
- Use common file methods: `open`, `read`, `readline`, `readlines`, `write`, `writelines`, `close`
- Learn about **context managers** (`with open(...)`) for safe file handling
- Explore the `os` module for:
  - Listing directories with `os.listdir()`
  - Walking directory trees with `os.walk()`
  - Creating directories with `os.makedirs()`
  - Deleting and renaming files with `os.remove()` and `os.rename()`
  - Checking file and directory existence with `os.path`
  - Getting file metadata with `os.stat()`
- Use **object-oriented file handling** with `pathlib.Path`
- Move and copy files with `shutil`
- Work with **temporary files** using `tempfile`
- Get a high-level overview of the **HTTP protocol**
- Use the `requests` library for simple HTTP GET and POST calls
- See how to register for **OpenRouter** and call an LLM API from Python

## Daily agenda and course flow

**09:00 - 10:30 (1h 30m)**
- Why work with files and APIs
- Basic file handling and common file methods
- Safe file handling with context managers

**10:30 - 10:45 (15m)**  
- Short break

**10:45 - 12:00 (1h 15m)**
- Directory handling with `os` and `os.path`
- Walking directories with `os.walk`
- Creating, renaming, deleting files and folders (`os`, `os.makedirs`)
- File metadata with `os.stat`

**12:00 - 13:00 (1h)**  
- Lunch break

**13:00 - 14:45 (1h 45m)**
- `pathlib` for object-oriented paths
- Moving and copying with `shutil`
- Temporary files with `tempfile`
- Short combined exercises

**14:45 - 15:00 (15m)**  
- Short break

**15:00 - 16:30 (1h 30m)**
- HTTP protocol overview
- `requests` basics
- OpenRouter registration and simple LLM call
- Complex combined example using file operations and HTTP
- Day summary and Q&A

### Helpful references

- File objects: https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files
- `os` module: https://docs.python.org/3/library/os.html
- `os.path`: https://docs.python.org/3/library/os.path.html
- `pathlib`: https://docs.python.org/3/library/pathlib.html
- `shutil`: https://docs.python.org/3/library/shutil.html
- `tempfile`: https://docs.python.org/3/library/tempfile.html
- HTTP basics (MDN): https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview
- `requests` library docs: https://requests.readthedocs.io/en/latest/
- OpenRouter: https://openrouter.ai/


# Recap – Objects, Classes, Inheritance and Argument Unpacking

These few cells contain a recap of several object oriented and function calling concepts in Python:

- objects, classes, instances and types
- attributes, fields, methods and `self`
- subclasses and inheritance
- `*` unpacking and `*args`
- `**` unpacking and `**kwargs`

## Topic 1 – Objects, classes, instances and types (recap)

### Knowledge: what these words mean

- An **object** is any value that lives in memory while your program runs. Numbers, strings, lists, and things you create with `class` are all objects.
- A **class** is a *blueprint* for creating objects of a certain kind. It describes what data they have and what they can do.
- An **instance** (often called an "object" in everyday speech) is one concrete thing created from a class. If `Person` is a class, then `Person("Anna")` creates one instance.
- A **type** is the kind of an object. For built in things like `5` the type is `int`. For objects you define, the type is your class.

Relations to earlier material:

- When you wrote `x = 5` or `text = "hi"`, you already worked with objects. You just did not call them that.
- `type(x)` is a function you already saw; it returns the class of the object stored in `x`.

When to think about these concepts:

- When you design a small model of the real world, like `Person`, `Order`, or `Invoice`, you are deciding which classes you need.
- When you see an error like `AttributeError: 'int' object has no attribute 'upper'`, it means you tried to use a string method on an object whose type is `int`.

Common confusion:

- "Is an object the same as an instance?" – In practice, yes; most people use the words as synonyms. More precisely: an instance is an object that was created from a class.
- "Is a class the same as a type?" – Almost. In modern Python, user defined types are implemented as classes, and `type(obj)` usually returns the class of `obj`.

Real world style example:

- A **class** `Car` describes what every car knows (brand, year) and can do (start, stop).
- Your actual **car** in the parking lot is one **instance** of that class.
- The **type** of that instance in Python is `<class 'Car'>`.


In [38]:
# Example: basic class, object (instance), and type

class Car:
    def __init__(self, brand, year):
        self.brand = brand
        self.year = year

    def describe(self):
        print(f"This car is a {self.year} {self.brand}.")

# Create two instances (objects) of the Car class
car1 = Car("Toyota", 2018)
car2 = Car("Tesla", 2022)

car1.describe()
car2.describe()

print("Type of car1:", type(car1))
print("Is car1 a Car?", isinstance(car1, Car))
print("Is 5 a Car?", isinstance(5, Car))

This car is a 2018 Toyota.
This car is a 2022 Tesla.
Type of car1: <class '__main__.Car'>
Is car1 a Car? True
Is 5 a Car? False


### Easy exercise – Model a simple device

**Task:**

- Create a class `Laptop` with two pieces of data:
  - `brand` (for example "Lenovo" or "Dell")
  - `ram_gb` (amount of RAM in gigabytes, for example `16`)
- Add a method `describe` that prints a short sentence, for example: `"This laptop is a Lenovo with 16 GB RAM."`
- Create **two instances** of `Laptop` with different data.
- Call `describe()` on both.
- Print the type of one instance using `type()`.

Focus on:

- understanding that `Laptop` is the class, and each laptop you create is an instance (object)
- seeing that `type(instance)` returns the class

In [ ]:
# Easy exercise starter: Laptop class

# TODO:
# 1. Define the Laptop class with __init__ and a describe method.
# 2. Create two instances.
# 3. Call describe() on both.
# 4. Print type() of one instance.

# class Laptop:
#     def __init__(self, brand, ram_gb):
#         ...
#
#     def describe(self):
#         ...
#
# laptop1 = ...
# laptop2 = ...
#
# laptop1.describe()
# laptop2.describe()
# print(type(laptop1))

In [39]:
# Easy exercise solution: Laptop class

class Laptop:
    def __init__(self, brand, ram_gb):
        self.brand = brand
        self.ram_gb = ram_gb

    def describe(self):
        print(f"This laptop is a {self.brand} with {self.ram_gb} GB RAM.")

laptop1 = Laptop("Lenovo", 16)
laptop2 = Laptop("Dell", 8)

laptop1.describe()
laptop2.describe()
print("Type of laptop1:", type(laptop1))

This laptop is a Lenovo with 16 GB RAM.
This laptop is a Dell with 8 GB RAM.
Type of laptop1: <class '__main__.Laptop'>


### Advanced exercise – Simple inventory with types

**Task:**

1. Create a class `Product` with:
   - `name` (string)
   - `price_huf` (integer or float)
2. Add a method `full_label` that returns a string like `"Notebook – 4999 HUF"`.
3. Create a list called `inventory` that contains several `Product` instances.
4. Iterate over `inventory` and:
   - print the `full_label()` of each product
   - assert that each item is really a `Product` using `isinstance(item, Product)` and print the result

This shows how classes, instances, and types work together in a slightly more realistic situation.

In [ ]:
# Advanced exercise starter: Product inventory

# TODO:
# 1. Define Product with name and price_huf.
# 2. Add full_label method.
# 3. Create a list of Product instances.
# 4. Loop and print labels and type checks.

# class Product:
#     def __init__(self, name, price_huf):
#         ...
#
#     def full_label(self):
#         ...
#
# inventory = [
#     # create some Product instances here
# ]
#
# for item in inventory:
#     ...  # print label and isinstance check

In [40]:
# Advanced exercise solution: Product inventory

class Product:
    def __init__(self, name, price_huf):
        self.name = name
        self.price_huf = price_huf

    def full_label(self):
        return f"{self.name} – {self.price_huf} HUF"

inventory = [
    Product("Notebook", 4999),
    Product("Mouse", 2990),
    Product("Keyboard", 7990),
]

for item in inventory:
    print(item.full_label())
    print("Is Product?", isinstance(item, Product))

Notebook – 4999 HUF
Is Product? True
Mouse – 2990 HUF
Is Product? True
Keyboard – 7990 HUF
Is Product? True


## Topic 2 – Attributes, fields, methods and `self`

### Knowledge: what lives inside an object

- An **attribute** is a named piece of data stored on an object, for example `account.balance`.
- A **field** is just another word people use for a data attribute. In Python, "attribute" is the common word; "field" is more common in database and form contexts.
- A **method** (or member function) is a function defined inside a class that works with the object, for example `account.deposit(1000)`.
- `self` is the **first parameter of a method** that refers to "this object". Python passes it automatically when you call the method.

Relations to earlier material:

- You already used variables, like `balance = 1000`. An attribute is similar, but it lives *inside* an object, for example `account.balance`.
- Functions you saw earlier took normal parameters. Methods are functions that are attached to a class and receive the instance as `self`.

Common confusion:

- "Do I have to name it `self`?" – Technically no, but everyone does. The name is a strong convention. The important part is that it is the first parameter.
- "Why do I write `self.balance` inside the class?" – Because inside the method you need to say which object you are reading or changing. `self.balance` means "the balance of this specific account".

Real world style example:

- A class `BankAccount` might have:
  - attributes / fields: `owner`, `balance`
  - methods: `deposit`, `withdraw`
  - inside `deposit`, you use `self.balance` to update that particular account, not all accounts.

In [41]:
# Example: attributes (fields), methods, and self

class BankAccount:
    def __init__(self, owner, initial_balance=0):
        self.owner = owner          # attribute / field
        self.balance = initial_balance  # attribute / field

    def deposit(self, amount):
        self.balance += amount      # use self to access this account

    def withdraw(self, amount):
        if amount <= self.balance:
            self.balance -= amount
        else:
            print("Not enough balance.")

    def show_summary(self):
        print(f"Owner: {self.owner}, balance: {self.balance} HUF")

account = BankAccount("Anna", 10000)
account.show_summary()
account.deposit(2500)
account.show_summary()
account.withdraw(20000)
account.show_summary()

Owner: Anna, balance: 10000 HUF
Owner: Anna, balance: 12500 HUF
Not enough balance.
Owner: Anna, balance: 12500 HUF


### Easy exercise – A simple counter object

**Task:**

- Create a class `Counter` with:
  - an attribute `value` that starts at `0`
- Add methods:
  - `increment()` that increases `value` by 1
  - `reset()` that sets `value` back to 0
  - `show()` that prints the current value
- Create one instance of `Counter` and call these methods in some order to see the changes.

Focus on:

- using `self.value` inside methods to read and update the internal state
- understanding that different instances would have their own `value`

In [ ]:
# Easy exercise starter: Counter class

# TODO:
# 1. Define Counter with value starting at 0.
# 2. Implement increment, reset, show methods using self.
# 3. Create an instance and call the methods.

# class Counter:
#     def __init__(self):
#         ...
#
#     def increment(self):
#         ...
#
#     def reset(self):
#         ...
#
#     def show(self):
#         ...
#
# c = Counter()
# c.show()
# c.increment()
# c.show()
# c.reset()
# c.show()

In [42]:
# Easy exercise solution: Counter class

class Counter:
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1

    def reset(self):
        self.value = 0

    def show(self):
        print(f"Counter value: {self.value}")

c = Counter()
c.show()
c.increment()
c.increment()
c.show()
c.reset()
c.show()

Counter value: 0
Counter value: 2
Counter value: 0


### Advanced exercise – Temperature sensor with attributes and methods

**Task:**

1. Create a class `TemperatureSensor` with attributes:
   - `location` (for example "Office" or "Server room")
   - `current_celsius` (float)
2. Add methods:
   - `set_temperature(new_celsius)` to update the `current_celsius`
   - `in_fahrenheit()` that returns the temperature in Fahrenheit (formula: `c * 9 / 5 + 32`)
   - `describe()` that prints something like: `"Office: 22.5 C (72.5 F)"`
3. Create a few instances and call these methods.

This is a realistic pattern: an object stores some state (the latest measurement) and has methods to work with that state.

In [ ]:
# Advanced exercise starter: TemperatureSensor

# TODO:
# 1. Define TemperatureSensor with location and current_celsius.
# 2. Add set_temperature, in_fahrenheit, describe methods.
# 3. Create a few sensors and call methods.

# class TemperatureSensor:
#     def __init__(self, location, current_celsius):
#         ...
#
#     def set_temperature(self, new_celsius):
#         ...
#
#     def in_fahrenheit(self):
#         ...
#
#     def describe(self):
#         ...
#
# sensor1 = TemperatureSensor("Office", 22.5)
# sensor2 = TemperatureSensor("Server room", 27.0)
#
# sensor1.describe()
# sensor2.describe()
# sensor1.set_temperature(23.0)
# sensor1.describe()

In [43]:
# Advanced exercise solution: TemperatureSensor

class TemperatureSensor:
    def __init__(self, location, current_celsius):
        self.location = location
        self.current_celsius = current_celsius

    def set_temperature(self, new_celsius):
        self.current_celsius = new_celsius

    def in_fahrenheit(self):
        return self.current_celsius * 9 / 5 + 32

    def describe(self):
        f = self.in_fahrenheit()
        print(f"{self.location}: {self.current_celsius:.1f} C ({f:.1f} F)")

sensor1 = TemperatureSensor("Office", 22.5)
sensor2 = TemperatureSensor("Server room", 27.0)

sensor1.describe()
sensor2.describe()

sensor1.set_temperature(23.0)
sensor1.describe()

Office: 22.5 C (72.5 F)
Server room: 27.0 C (80.6 F)
Office: 23.0 C (73.4 F)


## Topic 3 – Subclasses and inheritance

### Knowledge: reusing and specialising behavior

- A **subclass** is a class that *inherits* from another class (the **base class** or **superclass**).
- "Inheritance" means the subclass automatically gets the attributes and methods of the base class.
- The subclass can:
  - reuse existing behavior as is
  - add new attributes or methods
  - override methods to change behavior

Relations to earlier material:

- You already saw how one class can model a thing. Inheritance is about modelling a family of related things.

Examples of "is a" relationships:

- `Manager` is a kind of `Employee`.
- `SavingsAccount` is a kind of `BankAccount`.
- `ElectricCar` is a kind of `Car`.

Common confusion:

- "When should I use inheritance and when should I just add attributes?" – Very rough rule: use inheritance when one thing **is a** more specific version of another thing. Use attributes when one thing **has a** another thing (for example a `Car` has an `Engine`).

Real world style example:

- Base class `Employee` has `name` and `get_monthly_pay()`.
- Subclass `SalesEmployee` adds `commission` and overrides `get_monthly_pay()` to include it.

In [45]:
# Example: simple inheritance

class Employee:
    def __init__(self, name, base_salary):
        self.name = name
        self.base_salary = base_salary

    def monthly_pay(self):
        return self.base_salary

    def describe(self):
        print(f"Employee {self.name}, monthly pay: {self.monthly_pay()} HUF")

class SalesEmployee(Employee):
    def __init__(self, name, base_salary, commission):
        super().__init__(name, base_salary)  # call base class constructor
        self.commission = commission

    def monthly_pay(self):
        # override: pay includes base salary plus commission
        return self.base_salary + self.commission

anna = Employee("Anna", 400000)
ben = SalesEmployee("Ben", 350000, 80000)

anna.describe()
ben.describe()

print("Is ben an Employee?", isinstance(ben, Employee))
print("Is ben a SalesEmployee?", isinstance(ben, SalesEmployee))

Employee Anna, monthly pay: 400000 HUF
Employee Ben, monthly pay: 430000 HUF
Is ben an Employee? True
Is ben a SalesEmployee? True


### Easy exercise – Shapes with a base class

**Task:**

1. Create a base class `Shape` with:
   - an attribute `name`
   - a method `describe()` that prints `"Shape: <name>"`
2. Create a subclass `Rectangle` that:
   - adds attributes `width` and `height`
   - adds a method `area()` that returns `width * height`
   - overrides `describe()` to print something like: `"Rectangle 3 x 4, area: 12"`
3. Create a `Rectangle` instance and call both `area()` and `describe()`.

Focus on:

- using `super().__init__` to call the base class constructor
- seeing that the subclass can still use code from the base class

In [46]:
# Easy exercise starter: Shape and Rectangle

# TODO:
# 1. Define Shape with name and describe.
# 2. Define Rectangle(Shape) with width, height, area, and overridden describe.
# 3. Create an instance and test.

# class Shape:
#     def __init__(self, name):
#         ...
#
#     def describe(self):
#         ...
#
# class Rectangle(Shape):
#     def __init__(self, name, width, height):
#         ...
#
#     def area(self):
#         ...
#
#     def describe(self):
#         ...
#
# rect = Rectangle("My rectangle", 3, 4)
# print("Area:", rect.area())
# rect.describe()

In [47]:
# Easy exercise solution: Shape and Rectangle

class Shape:
    def __init__(self, name):
        self.name = name

    def describe(self):
        print(f"Shape: {self.name}")

class Rectangle(Shape):
    def __init__(self, name, width, height):
        super().__init__(name)
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def describe(self):
        print(f"Rectangle {self.width} x {self.height}, area: {self.area()}")

rect = Rectangle("My rectangle", 3, 4)
print("Area:", rect.area())
rect.describe()

Area: 12
Rectangle 3 x 4, area: 12


### Advanced exercise – Polymorphic area calculation

**Task:**

1. Extend the previous idea:
   - Keep the base class `Shape` as before.
   - Keep `Rectangle` as a subclass.
   - Add a new subclass `Circle` that stores a `radius` and has an `area()` method using `3.14 * radius * radius`.
2. Create a list called `shapes` that contains both `Rectangle` and `Circle` instances.
3. Write a function `total_area(shapes)` that:
   - takes a list of shapes
   - loops through them
   - calls `shape.area()` on each and sums up the result
4. Call `total_area(shapes)` and print the result.

This shows **polymorphism**: different subclasses implement the same method name (`area`), and you can call it without caring about the exact subclass.

In [ ]:
# Advanced exercise starter: multiple shape subclasses

# TODO:
# 1. Define Shape, Rectangle, Circle (if not already defined here).
# 2. Implement area() in both subclasses.
# 3. Implement total_area(shapes) function.
# 4. Create a list with both types and test.

# class Shape:
#     ...
#
# class Rectangle(Shape):
#     ...
#
# class Circle(Shape):
#     ...
#
# def total_area(shapes):
#     ...
#
# shapes = [
#     # some Rectangle and Circle instances
# ]
#
# print("Total area:", total_area(shapes))

In [48]:
# Advanced exercise solution: multiple shape subclasses

class Shape:
    def __init__(self, name):
        self.name = name

    def describe(self):
        print(f"Shape: {self.name}")

class Rectangle(Shape):
    def __init__(self, name, width, height):
        super().__init__(name)
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def describe(self):
        print(f"Rectangle {self.width} x {self.height}, area: {self.area()}")

class Circle(Shape):
    def __init__(self, name, radius):
        super().__init__(name)
        self.radius = radius

    def area(self):
        return 3.14 * self.radius * self.radius

    def describe(self):
        print(f"Circle radius {self.radius}, area: {self.area():.2f}")

def total_area(shapes):
    total = 0
    for s in shapes:
        total += s.area()
    return total

shapes = [
    Rectangle("R1", 3, 4),
    Rectangle("R2", 2, 5),
    Circle("C1", 2),
]

for s in shapes:
    s.describe()

print("Total area:", total_area(shapes))

Rectangle 3 x 4, area: 12
Rectangle 2 x 5, area: 10
Circle radius 2, area: 12.56
Total area: 34.56


## Topic 4 – `*` unpacking and `*args`

### Knowledge: flexible positional arguments

There are two related but different ideas here:

1. **Sequence unpacking with `*` in a function call**
   - If you have a list or tuple, `*` in front of it will "unpack" its elements into separate positional arguments.
   - Example: if `nums = [1, 2, 3]`, then `add(*nums)` means `add(1, 2, 3)` if `add` takes three parameters.

2. **Collecting extra positional arguments with `*args` in a function definition**
   - Inside a function definition, writing `*args` means: "collect any extra positional arguments into a tuple called `args`".
   - The name `args` is just a convention; the important part is the `*`.

Common confusion:

- "Is `args` a list or a tuple?" – It is a **tuple**. It behaves similarly to a list for iteration, but it is immutable.
- "Can I have other parameters together with `*args`?" – Yes. Typically you have some normal parameters, then `*args`.

Real world style example:

- A function `def sum_all(*numbers):` can be called as `sum_all(1, 2, 3)` or `sum_all(*my_list)`. In both cases, inside the function you just loop over `numbers`.

In [49]:
# Example: *args and * sequence unpacking

def sum_all(*numbers):
    print("numbers tuple inside function:", numbers)
    total = 0
    for n in numbers:
        total += n
    return total

print("Direct arguments:", sum_all(1, 2, 3))

values = [10, 20, 30]
# Use * to unpack the list into separate arguments
print("Unpacked list:", sum_all(*values))

# You can also use * in assignment
first, *middle, last = [1, 2, 3, 4, 5]
print("first:", first)
print("middle:", middle)
print("last:", last)

numbers tuple inside function: (1, 2, 3)
Direct arguments: 6
numbers tuple inside function: (10, 20, 30)
Unpacked list: 60
first: 1
middle: [2, 3, 4]
last: 5


### Easy exercise – Average of arbitrary many numbers

**Task:**

- Write a function `average(*numbers)` that:
  - accepts any number of numeric positional arguments
  - returns the average of those numbers
- Handle the case when no numbers are given:
  - for example, return `0` or print a message and return `None`
- Call the function in two ways:
  - directly, like `average(1, 2, 3)`
  - with a list using unpacking, like `average(*[10, 20, 30, 40])`

Focus on:

- understanding that inside the function, `numbers` is just a tuple
- practising both `*args` and `*` unpacking

In [ ]:
# Easy exercise starter: average with *args

# TODO:
# 1. Define average(*numbers).
# 2. Handle empty input.
# 3. Test with both direct arguments and an unpacked list.

# def average(*numbers):
#     ...
#
# print(average(1, 2, 3))
# values = [10, 20, 30, 40]
# print(average(*values))

In [50]:
# Easy exercise solution: average with *args

def average(*numbers):
    if not numbers:
        print("No numbers provided.")
        return 0
    total = 0
    for n in numbers:
        total += n
    return total / len(numbers)

print("Average of 1, 2, 3:", average(1, 2, 3))
values = [10, 20, 30, 40]
print("Average of list:", average(*values))

Average of 1, 2, 3: 2.0
Average of list: 25.0


### Advanced exercise – Flexible logging function

**Task:**

- Write a function `log_messages(prefix, *messages)` that:
  - `prefix` is a normal parameter, for example `"INFO"` or `"ERROR"`
  - `*messages` can be any number of string messages
  - for each message, print a line like: `"[INFO] Message text"`
- Call it several times, for example:
  - `log_messages("INFO", "Started", "Loading config", "Ready")`
  - `log_messages("ERROR", "Something went wrong")`

This pattern is useful when you do not know in advance how many things the caller wants to log.

In [ ]:
# Advanced exercise starter: log_messages with *args

# TODO:
# 1. Implement log_messages(prefix, *messages).
# 2. Loop through messages and print them with the prefix.
# 3. Call the function with different numbers of messages.

# def log_messages(prefix, *messages):
#     ...
#
# log_messages("INFO", "Started", "Loading config", "Ready")
# log_messages("ERROR", "Something went wrong")

In [51]:
# Advanced exercise solution: log_messages with *args

def log_messages(prefix, *messages):
    for msg in messages:
        print(f"[{prefix}] {msg}")

log_messages("INFO", "Started", "Loading config", "Ready")
log_messages("ERROR", "Something went wrong")

[INFO] Started
[INFO] Loading config
[INFO] Ready
[ERROR] Something went wrong


## Topic 5 – `**` unpacking and `**kwargs`

### Knowledge: flexible keyword arguments

Again there are two related ideas:

1. **Dictionary unpacking with `**` in a function call**
   - If you have a dictionary, `**` in front of it will pass its key value pairs as separate keyword arguments.
   - Example: if `data = {"name": "Anna", "age": 30}`, then `show_person(**data)` is the same as `show_person(name="Anna", age=30)`.

2. **Collecting extra keyword arguments with `**kwargs` in a function definition**
   - Inside a function definition, `**kwargs` means: "collect any extra keyword arguments into a dictionary called `kwargs`".
   - The name `kwargs` is a convention; the important part is the `**`.

Common confusion:

- "What type is `kwargs`?" – It is a **dictionary**.
- "Can I mix normal parameters, `*args`, and `**kwargs`?" – Yes. The usual order is: normal parameters, then `*args`, then `**kwargs`.

Real world style example:

- A configuration function might accept many optional settings:
  - `setup_db(host="localhost", port=5432, debug=True, timeout=30)`
  - Using `**kwargs`, the function can forward these options to another lower level function.


In [52]:
# Example: **kwargs and ** dict unpacking

def show_user_profile(**info):
    print("User profile:")
    for key, value in info.items():
        print(f"  {key}: {value}")

show_user_profile(name="Anna", age=30, city="Budapest")

user_dict = {"name": "Ben", "age": 25, "city": "Szeged"}

# Use ** to unpack the dictionary into keyword arguments
show_user_profile(**user_dict)

User profile:
  name: Anna
  age: 30
  city: Budapest
User profile:
  name: Ben
  age: 25
  city: Szeged


### Easy exercise – Describe a person with flexible details

**Task:**

- Write a function `describe_person(**info)` that:
  - prints a line `"Person details:"`
  - then loops through `info.items()` and prints each key value pair
- Call the function:
  - once with explicit keyword arguments, like `describe_person(name="Anna", age=30)`
  - once by unpacking a dictionary, like `describe_person(**person_data)`

Focus on:

- seeing that `info` is just a normal dictionary inside the function
- understanding the symmetry between `**` in the call and `**kwargs` in the definition

In [ ]:
# Easy exercise starter: describe_person with **kwargs

# TODO:
# 1. Implement describe_person(**info).
# 2. Call it with explicit keyword args and with an unpacked dict.

# def describe_person(**info):
#     ...
#
# describe_person(name="Anna", age=30)
# person_data = {"name": "Ben", "city": "Debrecen"}
# describe_person(**person_data)

In [53]:
# Easy exercise solution: describe_person with **kwargs

def describe_person(**info):
    print("Person details:")
    for key, value in info.items():
        print(f"  {key}: {value}")

describe_person(name="Anna", age=30)
person_data = {"name": "Ben", "city": "Debrecen"}
describe_person(**person_data)

Person details:
  name: Anna
  age: 30
Person details:
  name: Ben
  city: Debrecen


### Advanced exercise – URL builder with query parameters

**Task:**

1. Write a function `build_url(base, **params)` that:
   - `base` is a string like `"https://example.com/search"`
   - `**params` contains key value pairs that should go into the query string
2. The function should:
   - if `params` is empty, return `base`
   - otherwise, build a string like:
     - `"https://example.com/search?q=python&page=2"`
   - keys and values can be converted to strings with `str()`
   - join key value pairs with `&`
3. Call the function with different parameter combinations, for example:
   - `build_url("https://example.com/search", q="python", page=2)`
   - `build_url("https://example.com/report", year=2024, format="pdf")`

You do not need to handle URL escaping perfectly; this is just to practice working with `**kwargs` and dictionaries.

In [ ]:
# Advanced exercise starter: build_url with **params

# TODO:
# 1. Implement build_url(base, **params).
# 2. Construct the query string from params.items().
# 3. Test with different calls.

# def build_url(base, **params):
#     ...
#
# print(build_url("https://example.com/search", q="python", page=2))
# print(build_url("https://example.com/report", year=2024, format="pdf"))
# print(build_url("https://example.com/home"))

In [54]:
# Advanced exercise solution: build_url with **params

def build_url(base, **params):
    if not params:
        return base
    parts = []
    for key, value in params.items():
        parts.append(f"{key}={value}")
    query = "&".join(parts)
    return f"{base}?{query}"

print(build_url("https://example.com/search", q="python", page=2))
print(build_url("https://example.com/report", year=2024, format="pdf"))
print(build_url("https://example.com/home"))

https://example.com/search?q=python&page=2
https://example.com/report?year=2024&format=pdf
https://example.com/home


## Recap summary

In this notebook you:

- revisited **objects, classes, instances, and types** and saw how classes act as blueprints for objects
- practised using **attributes / fields, methods, and `self`** to store and manipulate object state
- created **subclasses** and used **inheritance** to reuse and specialise behavior, including a small polymorphic example with shapes
- learned how `*` works for **sequence unpacking** and how `*args` collects arbitrary positional arguments
- learned how `**` works for **dictionary unpacking** and how `**kwargs` collects arbitrary keyword arguments

All of these tools will appear again and again in real projects. The exercises here are small on purpose, so that you can focus on the ideas:

- what is inside an object (attributes, methods)
- how instances relate to their classes (types)
- how to write functions that accept flexible numbers of parameters (`*args`, `**kwargs`)

If any concept still feels fuzzy, try to modify the examples:

- add more attributes to the classes
- print `type()` and `isinstance()` results more often
- add extra arguments to your `*args` and `**kwargs` functions

Small experiments are the fastest way to build intuition.

---

## 1. Why work with files and APIs?

In almost every real-world Python project you will:

- **Read input** from files (CSV exports, logs, config files, reports)
- **Write output** to files (results, logs, generated documents)
- Talk to other systems over the network using **HTTP APIs** (web services, LLMs, payment providers, etc.)

You can think of files and APIs as two major ways your program connects to the outside world:

- Files: slower but often large, persistent data living on disk
- HTTP APIs: remote services that can answer questions, store data, or perform actions for you

Today we learn the basic tools for both.

### Trivia

- Under the hood, Python file objects wrap OS level file descriptors. Most operations eventually become system calls like `open`, `read`, `write` in the operating system.
- HTTP is a **text-based protocol**. Even when you send JSON, underneath it is just text sent over TCP.


## 2. Basic file handling and common methods

Python uses the built-in function `open()` to work with files.

```python
f = open("example.txt", "w", encoding="utf-8")
```

Important file modes:

- `'r'` - read (file must exist)
- `'w'` - write (overwrite or create)
- `'a'` - append (add to end, create if missing)
- `'rb'`, `'wb'` - binary read/write (images, non-text data)

Common file methods:

- `f.read()` - read the whole file as a single string
- `f.readline()` - read one line at a time
- `f.readlines()` - read all lines into a list of strings
- `f.write(text)` - write a string to the file (returns number of bytes/characters)
- `f.writelines(list_of_strings)` - write a list of strings
- `f.close()` - release the file resource

Using these correctly is essential to avoid data loss and file corruption.

Documentation: https://docs.python.org/3/tutorial/inputoutput.html#methods-of-file-objects


In [1]:
# Example: writing and reading a simple text file with common methods

filename = "day4_example_basic.txt"

# 1. Write some lines to the file
f = open(filename, "w", encoding="utf-8")
f.write("First line\n")
f.writelines(["Second line\n", "Third line\n"])
# Always close when done
f.close()

# 2. Read the whole file at once
f = open(filename, "r", encoding="utf-8")
contents = f.read()
print("--- read() output ---")
print(contents)
f.close()

# 3. Read line by line
f = open(filename, "r", encoding="utf-8")
print("--- readline() calls ---")
print(repr(f.readline()))
print(repr(f.readline()))
print(repr(f.readline()))
f.close()

# 4. Read all lines into a list
f = open(filename, "r", encoding="utf-8")
lines = f.readlines()
print("--- readlines() output ---")
print(lines)
f.close()


--- read() output ---
First line
Second line
Third line

--- readline() calls ---
'First line\n'
'Second line\n'
'Third line\n'
--- readlines() output ---
['First line\n', 'Second line\n', 'Third line\n']


### ✏ Exercise (easy): Save and load a short note

Create a small script that:

1. Asks the user for a short note using `input()`.
2. Opens a file called `my_note.txt` in write mode and writes the note into it (do not forget the newline).
3. Closes the file.
4. Opens `my_note.txt` again in read mode.
5. Reads the content with `read()` and prints it.

Use the methods shown above: `open`, `write`, `read`, `close`.


In [ ]:
# TODO: implement saving and loading a short note.

# note = input("Enter a short note: ")

# 1. Open my_note.txt for writing and save the note
# f = ...
# ...
# f.close()

# 2. Open my_note.txt for reading and print its contents
# f = ...
# ...
# f.close()


In [2]:
# Reference solution: saving and loading a short note

note = "This is a test note written on day 4."  # replace with input(...) for interactive use

# 1. Save the note
f = open("my_note.txt", "w", encoding="utf-8")
f.write(note + "\n")
f.close()

# 2. Load and print the note
f = open("my_note.txt", "r", encoding="utf-8")
loaded = f.read()
f.close()

print("Loaded note:")
print(loaded)


Loaded note:
This is a test note written on day 4.



## 3. Context managers recap: with open(...)

From Day 3 you already know context managers and the `with` statement. With files, this is the preferred pattern:

```python
with open("example.txt", "r", encoding="utf-8") as f:
    data = f.read()
    # use data
```

Advantages:

- The file is closed automatically when the `with` block ends.
- It works even if an exception is raised inside the block.

Internally, the file object implements special methods `__enter__` and `__exit__`, which make it a context manager.

Documentation: https://docs.python.org/3/reference/datamodel.html#context-managers


In [3]:
# Example: using with open for safe reading

filename = "day4_with_example.txt"

# Write something quickly
with open(filename, "w", encoding="utf-8") as f:
    f.write("Line 1\n")
    f.write("Line 2\n")

# Now read safely
with open(filename, "r", encoding="utf-8") as f:
    print("Contents of day4_with_example.txt:")
    for line in f:
        print(repr(line))


Contents of day4_with_example.txt:
'Line 1\n'
'Line 2\n'


---
# Short break (10:30-10:45)

---

## 4. Working with directories using os and os.listdir()

The `os` module provides functions for interacting with the operating system.

`os.listdir(path)` returns a list of entries in the given directory.

```python
import os
entries = os.listdir(".")  # current directory
for name in entries:
    print(name)
```

You can combine this with `os.path` to check which entries are files or directories.

Documentation:

- `os` module: https://docs.python.org/3/library/os.html
- `os.listdir`: https://docs.python.org/3/library/os.html#os.listdir


In [4]:
# Example: list current directory contents

import os

print("Entries in current directory:")
for name in os.listdir("."):
    print(" -", name)


Entries in current directory:
 - .ipynb_checkpoints
 - day1_python_intro.ipynb
 - day2_control_flow_and_collections.ipynb
 - day3_functions_errors_oop.ipynb
 - day4_example_basic.txt
 - day4_files_and_apis.ipynb
 - day4_with_example.txt
 - example.txt
 - my_note.txt
 - untitled.txt


### ✏ Exercise (easy): List all .txt files in the current directory

Write a script that:

1. Uses `os.listdir(".")` to get all entries in the current directory.
2. Filters them to only keep entries that end with `.txt`.
3. Prints the `.txt` filenames, one per line.

You can use basic string methods like `name.endswith(".txt")`.


In [ ]:
# TODO: list all .txt files in the current directory.

#import os

# for name in os.listdir("."):
#     # if name ends with .txt, print it
#     ...


In [5]:
# Reference solution: list all .txt files

import os

print(".txt files in current directory:")
for name in os.listdir("."):
    if name.endswith(".txt"):
        print(name)


.txt files in current directory:
day4_example_basic.txt
day4_with_example.txt
example.txt
my_note.txt
untitled.txt


## 5. Walking directory trees with os.walk()

`os.walk(top)` lets you traverse a directory tree.

It yields a 3-tuple `(dirpath, dirnames, filenames)` for each directory.

```python
import os
for dirpath, dirnames, filenames in os.walk("."):
    print("Directory:", dirpath)
    print("Subdirectories:", dirnames)
    print("Files:", filenames)
```

This is very useful for tasks like:

- Finding all Python files under a project
- Searching for specific filenames
- Computing statistics over a whole directory tree

Documentation: https://docs.python.org/3/library/os.html#os.walk


In [7]:
# Example: count all .ipynb files under the current directory

import os

ipynb_count = 0
for dirpath, dirnames, filenames in os.walk("."):
    for filename in filenames:
        if filename.endswith(".ipynb"):
            ipynb_count += 1

print("Number of .ipynb files under current directory:", ipynb_count)


Number of .ipynb files under current directory: 6


### ⚡ Exercise (advanced): Report file counts per directory

Using `os.walk(".")`, write a script that:

1. Iterates over all directories starting from the current directory.
2. For each directory, counts how many files it has (just `len(filenames)`).
3. Prints lines like:
   - `"./: 5 files"`
   - `"./subdir: 3 files"`

Use only `os.walk` and the basics shown above.


In [ ]:
# TODO: print how many files each directory contains.

#import os

# for dirpath, dirnames, filenames in os.walk("."):
#     # compute number of files and print dirpath and count
#     ...


In [8]:
# Reference solution: file counts per directory

import os

for dirpath, dirnames, filenames in os.walk("."):
    count = len(filenames)
    print(f"{dirpath}: {count} files")


.: 9 files
.\.ipynb_checkpoints: 3 files


## 6. Checking file and directory existence with os.path

`os.path` contains helpers for working with paths.

Useful functions:

- `os.path.exists(path)` - does this path exist at all?
- `os.path.isfile(path)` - is it an existing regular file?
- `os.path.isdir(path)` - is it an existing directory?
- `os.path.join(a, b)` - join path components in an OS independent way

```python
import os
path = "my_note.txt"
if os.path.exists(path):
    print("Path exists")
```

Documentation: https://docs.python.org/3/library/os.path.html


In [9]:
# Example: checking path types

import os

paths_to_check = ["my_note.txt", ".", "definitely_not_existing_123.txt"]

for path in paths_to_check:
    print(f"Checking {path!r}:")
    print("  exists:", os.path.exists(path))
    print("  is file:", os.path.isfile(path))
    print("  is dir:", os.path.isdir(path))


Checking 'my_note.txt':
  exists: True
  is file: True
  is dir: False
Checking '.':
  exists: True
  is file: False
  is dir: True
Checking 'definitely_not_existing_123.txt':
  exists: False
  is file: False
  is dir: False


### ✏ Exercise (easy): Safely read a file if it exists

Write a script that:

1. Asks the user for a filename.
2. Uses `os.path.exists` and `os.path.isfile` to check if it is an existing file.
3. If it is a file, opens it using `with open(..., "r", encoding="utf-8")` and prints the first line.
4. If not, prints a friendly message.

Use the patterns shown above and the context manager pattern from Day 3.


In [ ]:
# TODO: safely read a file if it exists.

#import os

# filename = input("Enter filename to read: ")

# if ...:  # check if file exists and is a file
#     with open(filename, "r", encoding="utf-8") as f:
#         # read and print first line
#         ...
# else:
#     print("File does not exist or is not a regular file.")


In [10]:
# Reference solution: safely read a file if it exists

import os

filename = "my_note.txt"  # replace with input(...) for interactive use

if os.path.exists(filename) and os.path.isfile(filename):
    with open(filename, "r", encoding="utf-8") as f:
        first_line = f.readline()
    print("First line:", first_line)
else:
    print("File does not exist or is not a regular file.")


First line: This is a test note written on day 4.



## 7. Deleting files with os.remove()

If you want to delete a file, you can use `os.remove(path)`.

```python
import os
os.remove("old_file.txt")
```

Be careful: this operation is permanent from Python's point of view. There is no built-in undo.

Documentation: https://docs.python.org/3/library/os.html#os.remove


In [11]:
# Example: create and then delete a file

import os

filename = "day4_delete_me.txt"

# Create the file
with open(filename, "w", encoding="utf-8") as f:
    f.write("To be deleted.\n")

print("Created", filename, "exists:", os.path.exists(filename))

# Delete it
os.remove(filename)
print("After deletion, exists:", os.path.exists(filename))


Created day4_delete_me.txt exists: True
After deletion, exists: False


### ✏ Exercise (easy): Ask before deleting

Write a script that:

1. Asks the user for a filename to delete.
2. Checks with `os.path.isfile` if it is an existing file.
3. If yes, asks for confirmation with `input("Are you sure? (y/n): ")`.
4. If the user types `"y"`, delete the file with `os.remove` and print a confirmation.
5. Otherwise, print that nothing was deleted.
6. If the path is not a file, print an error message.


In [ ]:
# TODO: implement safe delete with confirmation.

#import os

# filename = input("Enter filename to delete: ")

# if os.path.isfile(filename):
#     answer = input("Are you sure? (y/n): ")
#     if answer == "y":
#         # delete file
#         ...
#     else:
#         print("Nothing deleted.")
# else:
#     print("The given path is not an existing file.")


In [12]:
# Reference solution: safe delete with confirmation

import os

filename = "day4_delete_test.txt"  # replace with input(...) for interactive use

# create file for the demo
if not os.path.exists(filename):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("delete test\n")

if os.path.isfile(filename):
    answer = "y"  # replace with input("Are you sure? (y/n): ")
    if answer == "y":
        os.remove(filename)
        print("File deleted.")
    else:
        print("Nothing deleted.")
else:
    print("The given path is not an existing file.")


File deleted.


## 8. Renaming files with os.rename()

You can rename or move a file with `os.rename(src, dst)`.

```python
import os
os.rename("old_name.txt", "new_name.txt")
```

If `dst` includes a different directory, this effectively moves the file.

Documentation: https://docs.python.org/3/library/os.html#os.rename


In [13]:
# Example: rename a file

import os

old_name = "day4_rename_old.txt"
new_name = "day4_rename_new.txt"

# Create file if not exists
if not os.path.exists(old_name) and not os.path.exists(new_name):
    with open(old_name, "w", encoding="utf-8") as f:
        f.write("rename me\n")

if os.path.exists(old_name):
    os.rename(old_name, new_name)
    print(f"Renamed {old_name!r} to {new_name!r}")
else:
    print("Nothing to rename.")


Renamed 'day4_rename_old.txt' to 'day4_rename_new.txt'


### ✏ Exercise (easy): Create a backup copy name with .bak

Write a script that:

1. Asks the user for an existing filename.
2. Checks with `os.path.isfile` that it exists.
3. Builds a new name by adding `.bak` to the end (for example `config.txt` -> `config.txt.bak`).
4. Uses `os.rename` to rename the original file to the backup name.
5. Prints the old and new names.


In [ ]:
# TODO: rename a file to add .bak extension.

#import os

# filename = input("Enter filename to backup: ")

# if os.path.isfile(filename):
#     backup_name = filename + ".bak"
#     # rename to backup_name
#     ...
# else:
#     print("File does not exist.")


In [14]:
# Reference solution: rename a file to add .bak

import os

filename = "day4_backup_test.txt"  # replace with input(...) for interactive use

# create file for demo
if not os.path.exists(filename):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("backup test\n")

if os.path.isfile(filename):
    backup_name = filename + ".bak"
    os.rename(filename, backup_name)
    print(f"Renamed {filename!r} to {backup_name!r}")
else:
    print("File does not exist.")


Renamed 'day4_backup_test.txt' to 'day4_backup_test.txt.bak'


## 9. Creating nested directories with os.makedirs()

`os.makedirs(path, exist_ok=False)` creates intermediate directories as needed.

```python
import os
os.makedirs("logs/2025/11/17", exist_ok=True)
```

With `exist_ok=True`, it will not raise an error if the directory already exists.

Documentation: https://docs.python.org/3/library/os.html#os.makedirs


In [15]:
# Example: create a nested directory structure

import os

nested_dir = os.path.join("day4_output", "logs", "2025", "11", "17")
os.makedirs(nested_dir, exist_ok=True)

print("Created or confirmed directory:", nested_dir)
print("Exists:", os.path.isdir(nested_dir))


Created or confirmed directory: day4_output\logs\2025\11\17
Exists: True


### 🏃‍♂️ Exercise (medium): Create per-user folders

Write a script that:

1. Has a list of usernames, for example `users = ["anna", "bela", "csaba"]`.
2. For each username, creates a folder `day4_users/<username>/inbox` using `os.makedirs` with `exist_ok=True`.
3. Prints the full path for each created inbox directory.

Use `os.path.join` to build paths.


In [ ]:
# TODO: create per-user inbox directories.

#import os

# users = ["anna", "bela", "csaba"]

# base = "day4_users"

# for user in users:
#     inbox_path = os.path.join(base, user, "inbox")
#     # create the directory tree
#     ...
#     print("Created inbox:", inbox_path)


In [16]:
# Reference solution: per-user inbox directories

import os

users = ["anna", "bela", "csaba"]
base = "day4_users"

for user in users:
    inbox_path = os.path.join(base, user, "inbox")
    os.makedirs(inbox_path, exist_ok=True)
    print("Created inbox:", inbox_path)


Created inbox: day4_users\anna\inbox
Created inbox: day4_users\bela\inbox
Created inbox: day4_users\csaba\inbox


## 10. File metadata with os.stat()

`os.stat(path)` returns an object with various metadata:

- `st_size` - size in bytes
- `st_mtime` - last modification time (seconds since Unix epoch)
- `st_ctime` - creation time on some systems

```python
import os
info = os.stat("my_note.txt")
print(info.st_size)
```

Documentation: https://docs.python.org/3/library/os.html#os.stat


In [17]:
# Example: show size and modification time

import os
import time

filename = "my_note.txt"

if os.path.exists(filename):
    info = os.stat(filename)
    print("Size in bytes:", info.st_size)
    print("Last modified (raw):", info.st_mtime)
    print("Last modified (readable):", time.ctime(info.st_mtime))
else:
    print("File", filename, "does not exist.")


Size in bytes: 39
Last modified (raw): 1764102302.814053
Last modified (readable): Tue Nov 25 21:25:02 2025


### ⚡ Exercise (advanced): Find the largest file in a directory

Write a script that:

1. Asks the user for a directory path.
2. Uses `os.listdir` and `os.path.join` to iterate over entries in that directory.
3. For each entry that is a file, uses `os.stat` to get its size.
4. Finds the file with the largest size and prints its name and size.
5. If there are no files in the directory, print a message.


In [ ]:
# TODO: find the largest file in a directory.

#import os

# folder = input("Enter directory path: ")

# if not os.path.isdir(folder):
#     print("Not a directory.")
# else:
#     largest_name = None
#     largest_size = 0
#     for name in os.listdir(folder):
#         full_path = os.path.join(folder, name)
#         if os.path.isfile(full_path):
#             # get size with os.stat
#             ...
#     if largest_name is None:
#         print("No files in this directory.")
#     else:
#         print("Largest file:", largest_name, "with size", largest_size, "bytes")


In [18]:
# Reference solution: largest file in a directory

import os

folder = "."  # replace with input(...) for interactive use

if not os.path.isdir(folder):
    print("Not a directory.")
else:
    largest_name = None
    largest_size = 0
    for name in os.listdir(folder):
        full_path = os.path.join(folder, name)
        if os.path.isfile(full_path):
            size = os.stat(full_path).st_size
            if largest_name is None or size > largest_size:
                largest_name = name
                largest_size = size
    if largest_name is None:
        print("No files in this directory.")
    else:
        print("Largest file:", largest_name, "with size", largest_size, "bytes")


Largest file: day3_functions_errors_oop.ipynb with size 152567 bytes


---
# Lunch break (12:00-13:00)

---

## 11. Object-oriented paths with pathlib

The `pathlib` module provides an object-oriented way to handle paths.

Key concepts:

- `Path` objects represent file system paths.
- You can use `/` to join paths in an OS independent way.
- Methods like `.exists()`, `.is_file()`, `.is_dir()`, `.glob()` help you work with files.

```python
from pathlib import Path
base = Path(".")
for path in base.glob("*.txt"):
    print(path, "exists?", path.exists())
```

Documentation: https://docs.python.org/3/library/pathlib.html


In [19]:
# Example: list .txt files using pathlib

from pathlib import Path

base = Path(".")

print(".txt files with pathlib:")
for path in base.glob("*.txt"):
    print(" -", path, "file?", path.is_file())


.txt files with pathlib:
 - day4_example_basic.txt file? True
 - day4_rename_new.txt file? True
 - day4_with_example.txt file? True
 - example.txt file? True
 - my_note.txt file? True
 - untitled.txt file? True


### 🧪 Exercise (medium): Separate files and directories

Using `pathlib.Path`, write a script that:

1. Creates a `Path` object for the current directory.
2. Iterates over all entries using `.iterdir()`.
3. Collects two lists: one for files, one for directories.
4. Prints both lists.

Use the methods `.is_file()` and `.is_dir()`.


In [ ]:
# TODO: separate files and directories using pathlib.

#from pathlib import Path

# base = Path(".")
# files = []
# dirs = []

# for path in base.iterdir():
#     if path.is_file():
#         # add to files
#         ...
#     elif path.is_dir():
#         # add to dirs
#         ...

# print("Files:", files)
# print("Directories:", dirs)


In [20]:
# Reference solution: separate files and directories with pathlib

from pathlib import Path

base = Path(".")
files = []
dirs = []

for path in base.iterdir():
    if path.is_file():
        files.append(path.name)
    elif path.is_dir():
        dirs.append(path.name)

print("Files:", files)
print("Directories:", dirs)


Files: ['day1_python_intro.ipynb', 'day2_control_flow_and_collections.ipynb', 'day3_functions_errors_oop.ipynb', 'day4_backup_test.txt.bak', 'day4_example_basic.txt', 'day4_files_and_apis.ipynb', 'day4_rename_new.txt', 'day4_with_example.txt', 'example.txt', 'my_note.txt', 'untitled.txt']
Directories: ['.ipynb_checkpoints', 'day4_output', 'day4_users']


## 12. Moving and copying files with shutil

The `shutil` module provides high-level file operations.

Common functions:

- `shutil.copy(src, dst)` - copy file contents and permissions
- `shutil.move(src, dst)` - move a file or directory

```python
import shutil
shutil.copy("source.txt", "copy.txt")
shutil.move("copy.txt", "backup/copy.txt")
```

Documentation: https://docs.python.org/3/library/shutil.html


In [21]:
# Example: copy and move a file

import os
import shutil

os.makedirs("day4_shutil", exist_ok=True)

src = "day4_shutil_source.txt"
copy_target = os.path.join("day4_shutil", "copy.txt")
move_target = os.path.join("day4_shutil", "moved_copy.txt")

# create source file
with open(src, "w", encoding="utf-8") as f:
    f.write("content for shutil demo\n")

# copy to folder
shutil.copy(src, copy_target)
print("Copied to", copy_target)

# move inside folder
shutil.move(copy_target, move_target)
print("Moved to", move_target)


Copied to day4_shutil\copy.txt
Moved to day4_shutil\moved_copy.txt


### 🏃‍♂️ Exercise (medium): Simple backup directory

Write a script that:

1. Asks the user for a directory path `src_dir`.
2. Creates a backup directory called `src_dir + "_backup"` using `os.makedirs`.
3. Iterates over all files directly inside `src_dir` (ignore subdirectories).
4. Copies each file into the backup directory using `shutil.copy`.
5. Prints what was copied.

Use `os.listdir`, `os.path.isfile`, `os.path.join`, `os.makedirs`, and `shutil.copy`.


In [ ]:
# TODO: implement a simple backup directory copier.

#import os
#import shutil

# src_dir = input("Enter directory to backup: ")

# if not os.path.isdir(src_dir):
#     print("Not a directory.")
# else:
#     backup_dir = src_dir + "_backup"
#     os.makedirs(backup_dir, exist_ok=True)
#     for name in os.listdir(src_dir):
#         full_src = os.path.join(src_dir, name)
#         if os.path.isfile(full_src):
#             full_dst = os.path.join(backup_dir, name)
#             # copy file
#             ...
#             print(f"Copied {full_src!r} to {full_dst!r}")


In [22]:
# Reference solution: simple backup directory

import os
import shutil

src_dir = "day4_shutil"  # replace with input(...) for interactive use

if not os.path.isdir(src_dir):
    print("Not a directory.")
else:
    backup_dir = src_dir + "_backup"
    os.makedirs(backup_dir, exist_ok=True)
    for name in os.listdir(src_dir):
        full_src = os.path.join(src_dir, name)
        if os.path.isfile(full_src):
            full_dst = os.path.join(backup_dir, name)
            shutil.copy(full_src, full_dst)
            print(f"Copied {full_src!r} to {full_dst!r}")


Copied 'day4_shutil\\moved_copy.txt' to 'day4_shutil_backup\\moved_copy.txt'


## 13. Temporary files with tempfile

Sometimes you need a file only for a short time (for example, to store intermediate results). The `tempfile` module helps create such temporary files and directories.

Common functions:

- `tempfile.TemporaryFile()` - creates a temporary file object
- `tempfile.NamedTemporaryFile()` - creates a temp file with a real name on disk
- `tempfile.TemporaryDirectory()` - creates a temporary directory

These objects are usually used as context managers and clean up automatically.

```python
import tempfile
with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
    tmp.write("hello\n")
    tmp.seek(0)
    print(tmp.read())
```

Documentation: https://docs.python.org/3/library/tempfile.html


In [23]:
# Example: using NamedTemporaryFile

import tempfile

with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
    print("Temporary file name:", tmp.name)
    tmp.write("Temporary content\n")
    tmp.seek(0)
    data = tmp.read()
    print("Read back:")
    print(data)

print("After the with block, the temporary file is deleted.")


Temporary file name: C:\Users\gregk\AppData\Local\Temp\tmp_vdgx_d8
Read back:
Temporary content

After the with block, the temporary file is deleted.


### 🧪 Exercise (medium): Write and read a temporary log file

Using `tempfile.NamedTemporaryFile`, write a script that:

1. Creates a named temporary file in text mode.
2. Writes a few log lines like `"INFO: Start"`, `"INFO: Working"`, `"INFO: Done"`.
3. Seeks back to the beginning.
4. Reads all lines and prints them.

Do all of this inside a `with` block so the file is cleaned up automatically.


In [ ]:
# TODO: write and read a temporary log file.

#import tempfile

# with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
#     # write some log lines
#     ...
#     # go back to start
#     ...
#     # read and print
#     ...

# print("Temporary log file should now be deleted.")


In [24]:
# Reference solution: temporary log file

import tempfile

with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
    tmp.write("INFO: Start\n")
    tmp.write("INFO: Working\n")
    tmp.write("INFO: Done\n")
    tmp.seek(0)
    print("Log contents:")
    for line in tmp:
        print(line.strip())

print("Temporary log file should now be deleted.")


Log contents:
INFO: Start
INFO: Working
INFO: Done
Temporary log file should now be deleted.


---
# Short break (14:45-15:00)

---

## 14. Introduction to HTTP and web APIs

HTTP (Hypertext Transfer Protocol) is the basic protocol of the web.

Key ideas:

- A **client** (your Python script, a browser) sends a **request** to a **server**.
- The request has:
  - A method: `GET`, `POST`, `PUT`, `DELETE`, ...
  - A URL: for example `https://api.example.com/data`
  - Optional headers and body (for example JSON data)
- The server sends back a **response** with:
  - A status code: `200 OK`, `404 Not Found`, `500 Internal Server Error`, ...
  - Headers
  - Body (HTML, JSON, binary, ...)

In Python, we often use the `requests` library to work with HTTP APIs in a convenient way.

Good HTTP overview: https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview


## 15. Simple HTTP requests with the requests library

The `requests` library makes HTTP calls easy.

Basic GET request:

```python
import requests
response = requests.get("https://httpbin.io/get")
print(response.status_code)
print(response.text)
```

For JSON APIs:

```python
data = response.json()
```

Requests docs: https://requests.readthedocs.io/en/latest/


In [28]:
# Example: simple GET request to httpbin

import requests

url = "https://httpbin.io/get"
response = requests.get(url)

print("Status:", response.status_code)

# Print part of the JSON response
if response.headers.get("Content-Type", "").startswith("application/json"):
    data = response.json()
    print("You sent these headers:")
    for key in list(data.get("headers", {}).keys()):
        print(f"  {key}: {data['headers'][key]}")
else:
    print("Response body:")
    print(response.text[:200])


Status: 200
You sent these headers:
  Accept: ['*/*']
  Accept-Encoding: ['gzip, deflate, br, zstd']
  Connection: ['keep-alive']
  Host: ['httpbin.io']
  User-Agent: ['python-requests/2.32.4']


### ✏ Exercise (easy): Fetch your origin IP from httpbin

Use `requests.get` to call `https://httpbin.io/ip` and:

1. Check that the status code is 200.
2. Parse the JSON body with `.json()`.
3. Print the value of the `origin` field.

Hint: the JSON looks like `{"origin": "..."}`.


In [ ]:
# TODO: fetch and print your origin IP from httpbin.

#import requests

# url = "https://httpbin.io/ip"
# response = requests.get(url)

# if response.status_code == 200:
#     data = ...  # parse JSON
#     origin = ...
#     print("Origin IP:", origin)
# else:
#     print("Request failed with status", response.status_code)


In [29]:
# Reference solution: fetch origin IP from httpbin

import requests

url = "https://httpbin.io/ip"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    origin = data.get("origin")
    print("Origin IP:", origin)
else:
    print("Request failed with status", response.status_code)


Origin IP: 185.91.187.251:33345


## 16. OpenRouter registration (LLM API)

OpenRouter is a service that allows you to access various large language models via a unified HTTP API.

Typical steps to use it from Python:

1. Go to https://openrouter.ai/ and create an account.
2. Generate an API key in your account settings.
3. Store that API key securely (for example, in an environment variable like `OPENROUTER_API_KEY`).
4. Use `requests.post` to call the `/api/v1/chat/completions` endpoint.

Never hard-code real API keys into code that will be shared publicly or committed to version control.


## 17. OpenRouter LLM call with requests

Here is a minimal example of calling an LLM on OpenRouter.
We assume you have set an environment variable `OPENROUTER_API_KEY` with your secret key.

```python
import os
import requests

api_key = os.environ.get("OPENROUTER_API_KEY")

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

payload = {
    "model": "openai/gpt-4.1-mini",  # or another supported model
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say hello from a Python script."},
    ],
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)
data = response.json()
print(data["choices"][0]["message"]["content"])
```

OpenRouter API docs: https://openrouter.ai/docs/api-reference/chat-completions


In [31]:
# Example: helper function to call OpenRouter (will only work if you have an API key set)

import os
import requests


def call_openrouter(prompt):
    """Call OpenRouter with a simple prompt and return the text response.

    Requires the environment variable OPENROUTER_API_KEY to be set.
    """
    api_key = "sk-or-v1-YOUR_API_KEY_HERE" # os.environ.get("OPENROUTER_API_KEY")
    if not api_key:
        print("OPENROUTER_API_KEY is not set. Skipping real API call.")
        return "(no response - missing API key)"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "openai/gpt-oss-20b:free",
        "messages": [
            {"role": "system", "content": "You are a concise assistant."},
            {"role": "user", "content": prompt},
        ],
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    data = response.json()
    return data["choices"][0]["message"]["content"]


if __name__ == "__main__":
    result = call_openrouter("Give me one short sentence about Python.")
    print("OpenRouter response:")
    print(result)


OpenRouter response:
Python is a versatile, beginner‑friendly programming language.


### 💪 Exercise (advanced): Ask the LLM to summarize a short text

Using the `call_openrouter(prompt)` helper (or a similar one you write), create a small script that:

1. Asks the user for a short paragraph of text.
2. Builds a prompt like: `"Summarize this in one sentence: <user text>"`.
3. Passes the prompt to `call_openrouter`.
4. Prints the returned summary.

If you do not have an API key, you can still implement the logic and then verify it later when you get access.


In [ ]:
# TODO: ask the user for text and summarize with OpenRouter.

# def summarize_text_with_llm(text):
#     prompt = "Summarize this in one extremely short sentence: " + text
#     # use call_openrouter(prompt)
#     ...

user_text = """Python is a highly versatile and widely adopted programming language that serves
as a foundation for countless areas of software development. It is commonly used in web development,
where frameworks like Django and Flask power everything from small websites to large-scale platforms.
In the field of data science and machine learning, Python has become the dominant language thanks to
libraries such as NumPy, pandas, TensorFlow, and PyTorch. It is also frequently chosen for
automation and scripting, enabling developers and engineers to streamline repetitive tasks and
integrate complex systems with minimal effort. Beyond these areas, Python’s readable syntax and
extensive ecosystem make it suitable for education, scientific computing, APIs, prototyping, and a
wide variety of general-purpose programming tasks."""  # replace with input(...)
# summary = summarize_text_with_llm(user_text)
# print("Summary:", summary)


In [37]:
# Reference solution: ask user text and summarize with OpenRouter

# We assume call_openrouter is already defined in this notebook.


def summarize_text_with_llm(text):
    prompt = "Summarize this in one extremely short sentence: " + text
    return call_openrouter(prompt)


user_text = """Python is a highly versatile and widely adopted programming language that serves
as a foundation for countless areas of software development. It is commonly used in web development,
where frameworks like Django and Flask power everything from small websites to large-scale platforms.
In the field of data science and machine learning, Python has become the dominant language thanks to
libraries such as NumPy, pandas, TensorFlow, and PyTorch. It is also frequently chosen for
automation and scripting, enabling developers and engineers to streamline repetitive tasks and
integrate complex systems with minimal effort. Beyond these areas, Python’s readable syntax and
extensive ecosystem make it suitable for education, scientific computing, APIs, prototyping, and a
wide variety of general-purpose programming tasks."""  # replace with input(...)

summary = summarize_text_with_llm(user_text)
print("Summary:", summary)


Summary: Python powers web, data science, automation, and many more tasks.


---

## Working with CSV/TSV measurement files (input → process → output)

Scientific and engineering work often produces **tabular measurement data** in CSV/TSV format:

- Small spectral datasets: `wavelength_nm, reflectance`, or ellipsometry `wavelength_nm, psi_deg, delta_deg`
- Electrical characterization tables: `voltage_V, current_A` or `voltage_V, capacitance_F`
- Thin-film thickness or sheet-resistance maps: 2D grids measured across a wafer or sample

In practice you often need to:

1. **Read** the data from a CSV/TSV file (input)
2. **Process** it: filter by wavelength, compute averages, min/max, simple derived quantities
3. **Write** the processed results to a new CSV/TSV (output) for later plotting or reporting

This pattern should come to mind whenever you receive a measurement export from an instrument and you want to:

- Clean it up (drop invalid rows)
- Restrict it to a wavelength/voltage range
- Compute simple summary metrics (mean, max, etc.)
- Save a smaller, more focused dataset for plotting in another tool (Excel, Origin, Python plotting code)

Below we look at a compact example using **basic file I/O and manual parsing** (splitting lines into columns),
then you will practice on small synthetic measurement tables that resemble real spectral, I/V, and thickness-map data.

In [1]:
# Working example: filter a simple spectral CSV and write a new CSV
from pathlib import Path

# 1) Create a tiny example CSV file (normally this would come from an instrument)
input_path = Path("spectral_measurement.csv")
rows = [
    {"wavelength_nm": 400, "reflectance": 0.32},
    {"wavelength_nm": 450, "reflectance": 0.35},
    {"wavelength_nm": 500, "reflectance": 0.38},
    {"wavelength_nm": 550, "reflectance": 0.40},
    {"wavelength_nm": 600, "reflectance": 0.42},
    {"wavelength_nm": 650, "reflectance": 0.39},
]

with input_path.open("w", encoding="utf-8") as f:
    # write header
    f.write("wavelength_nm,reflectance\n")
    # write each row as comma-separated values
    for row in rows:
        f.write(f"{row['wavelength_nm']},{row['reflectance']}\n")

# 2) Read the CSV, keep only wavelengths between 450 and 600 nm, compute average reflectance
filtered_rows = []

with input_path.open("r", encoding="utf-8") as f:
    lines = f.readlines()
    # first line is the header, skip it
    for line in lines[1:]:
        line = line.strip()
        if not line:
            continue
        parts = line.split(",")
        wl = float(parts[0])
        refl = float(parts[1])
        if 450 <= wl <= 600:
            filtered_rows.append({"wavelength_nm": wl, "reflectance": refl})

if filtered_rows:
    avg_refl = sum(r["reflectance"] for r in filtered_rows) / len(filtered_rows)
else:
    avg_refl = float("nan")

print(f"Kept {len(filtered_rows)} points between 450 and 600 nm.")
print(f"Average reflectance in this range: {avg_refl:.3f}")

# 3) Write the filtered data into a new CSV file using basic file I/O
output_path = Path("spectral_measurement_filtered.csv")
with output_path.open("w", encoding="utf-8") as f:
    f.write("wavelength_nm,reflectance\n")
    for row in filtered_rows:
        f.write(f"{row['wavelength_nm']},{row['reflectance']}\n")

print(f"Filtered data written to: {output_path.resolve()}")

Kept 4 points between 450 and 600 nm.
Average reflectance in this range: 0.388
Filtered data written to: C:\Users\gregk\Desktop\winpython\WPy64-31700\notebooks\python_beginner_course-main\spectral_measurement_filtered.csv


### ✏️ Exercise 1 (easy) – Basic spectral CSV processing

You receive a small spectral export with columns `wavelength_nm` and `reflectance`.

**Task:**
1. Use the provided `csv_text` with a few rows of data.
2. Save it into a file (e.g. `exercise1_spectral.csv`) and read it back using basic file I/O (`open`, `for` over lines).
3. Compute and print:
   - Minimum reflectance and its wavelength
   - Maximum reflectance and its wavelength

Hints:
- Convert numeric fields with `float()`.
- Use `splitlines()` or iterate over the file line by line and `split(',')` to get the columns.
- Track the current min/max in variables as you iterate.

This mirrors the common task: *"What is the best and worst reflectance in this measured range?"*

In [ ]:
# Exercise 1 starter

csv_text = """wavelength_nm,reflectance
400,0.30
450,0.35
500,0.37
550,0.41
600,0.39
"""

# TODO:
# 1) Save csv_text into a file (e.g. "exercise1_spectral.csv").
# 2) Open the file for reading with open(...).
# 3) Skip the header line, then for each remaining line:
#    - strip the line, split by comma
#    - convert wavelength_nm and reflectance to float
#    - update min and max reflectance and remember the corresponding wavelengths.
# 4) Print the results.

# with open("exercise1_spectral.csv", "w", encoding="utf-8") as f:
#     f.write(csv_text)

# min_refl = ...
# min_wl = ...
# max_refl = ...
# max_wl = ...

# with open("exercise1_spectral.csv", "r", encoding="utf-8") as f:
#     lines = f.readlines()
#     for line in lines[1:]:  # skip header
#         line = line.strip()
#         if not line:
#             continue
#         parts = line.split(",")
#         wl = ...   # float(parts[0])
#         refl = ... # float(parts[1])
#         # update min and max here

# print(f"Minimum reflectance: {min_refl} at {min_wl} nm")
# print(f"Maximum reflectance: {max_refl} at {max_wl} nm")

In [2]:
# Exercise 1 solution

csv_text = """wavelength_nm,reflectance
400,0.30
450,0.35
500,0.37
550,0.41
600,0.39
"""

# 1) Save the text into a file
file_path = "exercise1_spectral.csv"
with open(file_path, "w", encoding="utf-8") as f:
    f.write(csv_text)

# 2) Read the file and compute min/max
min_refl = float("inf")
min_wl = None
max_refl = float("-inf")
max_wl = None

with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines[1:]:  # skip header
        line = line.strip()
        if not line:
            continue
        parts = line.split(",")
        wl = float(parts[0])
        refl = float(parts[1])
        if refl < min_refl:
            min_refl = refl
            min_wl = wl
        if refl > max_refl:
            max_refl = refl
            max_wl = wl

print(f"Minimum reflectance: {min_refl:.3f} at {min_wl} nm")
print(f"Maximum reflectance: {max_refl:.3f} at {max_wl} nm")

Minimum reflectance: 0.300 at 400.0 nm
Maximum reflectance: 0.410 at 550.0 nm


### ✏️ Exercise 2 (medium) – Approximate resistance from I–V data

Now you have an **electrical I–V measurement** with columns `voltage_V` and `current_mA`.
At small voltages, the curve is almost linear and you want a quick estimate of resistance.

**Task:**
1. Use the given `iv_text` CSV (voltage in volts, current in milliamps).
2. Save it to a file (e.g. `exercise2_iv.csv`) and read it back using basic file I/O.
3. For each row, convert current from mA to A.
4. For rows where `voltage_V` is between 0.0 and 0.2 V (inclusive), compute an average resistance:
   - `R = voltage_V / current_A`
5. Print the average resistance in ohms.

This is similar to: *"Estimate the device resistance around zero bias from a small-signal I–V measurement."*

In [ ]:
# Exercise 2 starter

iv_text = """voltage_V,current_mA
0.00,0.0
0.05,0.52
0.10,1.05
0.15,1.50
0.20,1.95
0.30,3.10
"""

# TODO:
# 1) Save iv_text into a file (e.g. "exercise2_iv.csv").
# 2) Open the file and read all lines.
# 3) Skip the header. For each remaining line:
#    - split by comma, convert voltage_V to float V, current_mA to float
#    - convert mA to A (divide by 1000)
#    - for rows with 0.0 <= V <= 0.2 and nonzero current, compute R = V / I and collect into a list.
# 4) Compute and print the average R.

# file_path = "exercise2_iv.csv"
# with open(file_path, "w", encoding="utf-8") as f:
#     f.write(iv_text)

# resistances = []
# with open(file_path, "r", encoding="utf-8") as f:
#     lines = f.readlines()
#     for line in lines[1:]:
#         line = line.strip()
#         if not line:
#             continue
#         parts = line.split(",")
#         V = ...       # float(parts[0])
#         I_mA = ...    # float(parts[1])
#         I = ...       # convert mA to A
#         # if V is in [0.0, 0.2] and I is not zero, compute R and append

# if resistances:
#     avg_R = ...   # sum(...) / len(...)
#     print(f"Average resistance near 0 V: {avg_R} ohm")
# else:
#     print("No data in the selected voltage range.")

In [3]:
# Exercise 2 solution

iv_text = """voltage_V,current_mA
0.00,0.0
0.05,0.52
0.10,1.05
0.15,1.50
0.20,1.95
0.30,3.10
"""

# 1) Save the text into a file
file_path = "exercise2_iv.csv"
with open(file_path, "w", encoding="utf-8") as f:
    f.write(iv_text)

# 2) Read and compute approximate resistance
resistances = []
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines[1:]:  # skip header
        line = line.strip()
        if not line:
            continue
        parts = line.split(",")
        V = float(parts[0])
        I_mA = float(parts[1])
        I = I_mA / 1000.0  # convert mA to A
        if 0.0 <= V <= 0.2 and I != 0.0:
            R = V / I
            resistances.append(R)

if resistances:
    avg_R = sum(resistances) / len(resistances)
    print(f"Average resistance near 0 V: {avg_R:.1f} ohm")
else:
    print("No data in the selected voltage range.")

Average resistance near 0 V: 98.5 ohm


### ⚡ Exercise 3 (advanced) – Thickness map from TSV and CSV summary

Sometimes a tool measures **thickness across a sample** as a 2D map (rows and columns of thickness values).
The instrument might export this as a **TSV** (tab-separated values) grid without headers.

Example (each number is thickness in nm at a different position):

```text
98.5\t100.2\t101.0
97.8\t99.9\t100.5
96.9\t98.7\t99.8
```

**Task:**
1. Use `tsv_text` containing a 3×3 thickness map (tab-separated, no header).
2. Save it into a file (e.g. `thickness_map.tsv`) and read it using basic file I/O.
3. Convert all values to floats and collect them into a list.
4. Compute:
   - Average thickness
   - Minimum and maximum thickness
5. Write a **new CSV file** called `thickness_summary.csv` with one header row and one data row:
   - Columns: `average_thickness_nm, min_thickness_nm, max_thickness_nm`
6. Print the computed values so you can quickly see them in the notebook.

This is close to the real workflow: *"Read a thickness map TSV, compute basic statistics, and store a compact summary CSV for reporting."*

In [ ]:
# Exercise 3 starter
from pathlib import Path

tsv_text = """98.5	100.2	101.0
97.8	99.9	100.5
96.9	98.7	99.8
"""

# TODO:
# 1) Save tsv_text into a file (e.g. "thickness_map.tsv").
# 2) Open the file, read each line, and split by "\t".
# 3) Convert all values to floats and collect into a single list.
# 4) Compute average, min, max.
# 5) Write them into thickness_summary.csv as one row with header.
# 6) Print the values.

# tsv_path = Path("thickness_map.tsv")
# with tsv_path.open("w", encoding="utf-8") as f:
#     f.write(tsv_text)

# values = []
# with tsv_path.open("r", encoding="utf-8") as f:
#     for line in f:
#         line = line.strip()
#         if not line:
#             continue
#         parts = line.split("\t")
#         for cell in parts:
#             # convert cell to float and append to values
#             ...

# if values:
#     avg_th = ...
#     min_th = ...
#     max_th = ...
# else:
#     avg_th = min_th = max_th = float("nan")

# output_path = Path("thickness_summary.csv")
# with output_path.open("w", encoding="utf-8") as f_out:
#     # write header
#     # write one row with avg_th, min_th, max_th (comma-separated)
#     ...

# print(f"Average thickness: {avg_th} nm")
# print(f"Min thickness: {min_th} nm")
# print(f"Max thickness: {max_th} nm")
# print(f"Summary written to: {output_path.resolve()}")

In [4]:
# Exercise 3 solution
from pathlib import Path

tsv_text = """98.5	100.2	101.0
97.8	99.9	100.5
96.9	98.7	99.8
"""

# 1) Save TSV text into a file
tsv_path = Path("thickness_map.tsv")
with tsv_path.open("w", encoding="utf-8") as f:
    f.write(tsv_text)

# 2) Read TSV and collect values
values = []
with tsv_path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        parts = line.split("\t")
        for cell in parts:
            if cell.strip():
                values.append(float(cell))

if values:
    avg_th = sum(values) / len(values)
    min_th = min(values)
    max_th = max(values)
else:
    avg_th = min_th = max_th = float("nan")

# 3) Write summary CSV with basic file I/O
output_path = Path("thickness_summary.csv")
with output_path.open("w", encoding="utf-8") as f_out:
    f_out.write("average_thickness_nm,min_thickness_nm,max_thickness_nm\n")
    f_out.write(f"{avg_th},{min_th},{max_th}\n")

print(f"Average thickness: {avg_th:.2f} nm")
print(f"Min thickness: {min_th:.2f} nm")
print(f"Max thickness: {max_th:.2f} nm")
print(f"Summary written to: {output_path.resolve()}")

Average thickness: 99.26 nm
Min thickness: 96.90 nm
Max thickness: 101.00 nm
Summary written to: C:\Users\gregk\Desktop\winpython\WPy64-31700\notebooks\python_beginner_course-main\thickness_summary.csv


## 18. Complex combined example: Daily notes archiver

In this final example we combine concepts from previous days:

- Input and output with `input()` and `print()` (Day 1)
- Lists, loops, comprehensions (Day 2)
- Functions and error handling (Day 3)
- File and directory operations, metadata, and paths (Day 4)

### Task

Create a small **Daily notes archiver** that:

1. Uses `pathlib.Path` to work in a base directory called `notes`.
2. Asks the user for a note category, for example `"work"` or `"personal"`.
3. Creates a subdirectory `notes/<category>` if it does not exist.
4. Asks the user for a note text and saves it into a new file with a name based on the current timestamp, for example `note_20251117_153045.txt`.
5. After saving, lists all note files in that category directory using `.glob("*.txt")`.
6. For each note file, prints the filename and its size in bytes (using `os.stat` or `Path.stat()`).
7. Optionally (bonus): if there are more than 5 note files, move the oldest ones into an `archive` subdirectory using `shutil.move`.

Use functions where it makes sense, and handle errors (for example, invalid category names or file issues) with try/except if you want to practice.


In [ ]:
# TODO: implement the Daily notes archiver.

#from pathlib import Path
#import os
#import shutil
#import time

# def get_timestamp_string():
#     # return a string like 20251117_153045
#     ...

# def ensure_category_dir(base, category):
#     # create notes/<category> if it does not exist, return the Path
#     ...

# def save_note(category_dir, text):
#     # create a new file with timestamped name and write the note
#     ...

# def list_notes_with_sizes(category_dir):
#     # list all .txt files and print their size
#     ...

# def maybe_archive_old_notes(category_dir, max_files=5):
#     # bonus: move oldest files to category_dir / "archive" if there are more than max_files
#     ...

# def main():
#     base = Path("notes")
#     base.mkdir(exist_ok=True)
#     category = input("Enter note category (e.g. work, personal): ").strip()
#     if not category:
#         print("Empty category, aborting.")
#         return
#     category_dir = ensure_category_dir(base, category)
#     text = input("Enter your note: ")
#     save_note(category_dir, text)
#     print("Notes in this category:")
#     list_notes_with_sizes(category_dir)
#     # bonus
#     # maybe_archive_old_notes(category_dir)

# if __name__ == "__main__":
#     main()


In [36]:
# Reference solution: Daily notes archiver

from pathlib import Path
import os
import shutil
import time


def get_timestamp_string():
    """Return a timestamp string like 20251117_153045."""
    return time.strftime("%Y%m%d_%H%M%S")


def ensure_category_dir(base, category):
    """Create notes/<category> if needed and return its Path."""
    category_dir = base / category
    category_dir.mkdir(parents=True, exist_ok=True)
    return category_dir


def save_note(category_dir, text):
    """Save a note to a new timestamped file in category_dir."""
    timestamp = get_timestamp_string()
    filename = f"note_{timestamp}.txt"
    path = category_dir / filename
    with path.open("w", encoding="utf-8") as f:
        f.write(text + "\n")
    print("Saved note to", path)
    return path


def list_notes_with_sizes(category_dir):
    """Print all note files and their sizes in bytes."""
    files = sorted(category_dir.glob("*.txt"))
    if not files:
        print("No notes yet.")
        return files
    for path in files:
        size = path.stat().st_size
        print(f"{path.name}: {size} bytes")
    return files


def maybe_archive_old_notes(category_dir, max_files=5):
    """If there are more than max_files notes, move the oldest ones to archive/.

    Oldest is determined by modification time.
    """
    files = list(category_dir.glob("*.txt"))
    if len(files) <= max_files:
        return
    files.sort(key=lambda p: p.stat().st_mtime)  # oldest first
    archive_dir = category_dir / "archive"
    archive_dir.mkdir(exist_ok=True)
    to_archive = files[:-max_files]
    for path in to_archive:
        target = archive_dir / path.name
        shutil.move(str(path), str(target))
        print("Archived", path.name, "to", target)


def main():
    base = Path("notes")
    base.mkdir(exist_ok=True)

    category = "demo"  # replace with input("Enter note category (e.g. work, personal): ").strip()
    if not category:
        print("Empty category, aborting.")
        return

    category_dir = ensure_category_dir(base, category)

    text = "This is a demo note."  # replace with input("Enter your note: ")
    save_note(category_dir, text)

    print("Notes in this category:")
    files = list_notes_with_sizes(category_dir)

    # bonus: archive if many notes
    maybe_archive_old_notes(category_dir, max_files=5)


if __name__ == "__main__":
    main()


Saved note to notes\demo\note_20251125_214713.txt
Notes in this category:
note_20251125_214713.txt: 22 bytes


## Day 4 summary

Today you learned how to:

- Use common file methods: `open`, `read`, `readline`, `readlines`, `write`, `writelines`, `close`
- Use `with open(...)` context managers for safe file handling
- List directories and traverse directory trees with `os.listdir` and `os.walk`
- Create nested directories with `os.makedirs`
- Delete and rename files with `os.remove` and `os.rename`
- Check file and directory existence with `os.path.exists`, `os.path.isfile`, `os.path.isdir`
- Inspect file metadata (size, modification time) with `os.stat`
- Use `pathlib.Path` for object-oriented path handling, `.exists()`, `.is_file()`, `.is_dir()`, `.glob()`
- Copy and move files with `shutil.copy` and `shutil.move`
- Create and use temporary files with `tempfile.NamedTemporaryFile`
- Understand the basics of HTTP: requests, responses, methods, status codes
- Use the `requests` library for simple GET calls and parse JSON responses
- Understand how to register for OpenRouter and how to call an LLM API with `requests.post`
- Combine file, directory, and API concepts in a small Daily notes archiver project

These tools are fundamental for many practical Python tasks: processing data files, building automation scripts, and integrating with web services and LLMs.
